<h1>Reconocimiento de audio</h1>

<b>Disclaimer: Este notebook utiliza funciones que estan definidos en distintos archivos .py que estan incluidos en la carpeta del proyecto <b>

<h2>Importacion de librerias necesarias</h2>

In [2375]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from tensorflow.keras.regularizers import l2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import parselmouth
from audioLoading import loadAudios
from scipy.io import wavfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from convertFiles import convertFiles
from pathlib import Path
from scipy.io import wavfile

Cargar los audios

In [1944]:
audios = loadAudios(f"./AUDIO/All-Audios/Metodo-01_Denoised/*")
audios_features = []

In [1924]:
print(audios)

[[(array([-1.3313080e-05, -9.3354274e-06, -9.7812981e-06, ...,
       -2.0399716e-04, -2.0335178e-04, -8.2252896e-05], dtype=float32), 22050), 'AdriaM-03_1.mp3'], [(array([-1.5522096e-07, -2.0305548e-07, -2.5823735e-07, ...,
       -6.2920004e-03, -6.8021575e-03, -2.8185856e-03], dtype=float32), 22050), 'AdriaM-03_2.mp3'], [(array([-5.2697473e-08, -1.5160907e-07, -2.0447078e-07, ...,
        1.6487477e-04,  1.4839305e-04,  5.5636774e-05], dtype=float32), 22050), 'AdriaM-03_3.mp3'], [(array([-2.4270494e-07, -2.5347794e-07, -1.6956022e-07, ...,
       -3.4186192e-04, -2.0765293e-04, -4.9369781e-05], dtype=float32), 22050), 'AdriaM-03_4.mp3'], [(array([7.8834290e-08, 1.0038820e-07, 9.6849071e-08, ..., 7.1847008e-04,
       6.8352307e-04, 2.6229228e-04], dtype=float32), 22050), 'AdriaM-03_5.mp3'], [(array([ 0.        ,  0.        ,  0.        , ..., -0.00118277,
        0.00132469,  0.00157111], dtype=float32), 22050), 'AleM-03_1.mp3'], [(array([ 0.        ,  0.        ,  0.        , ..., 

Extraccion de caracteristicas

In [2403]:
audios_features = []
labels =[]
for audio_signal,filename in audios:
    features = [
        np.mean(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13)[0,0]),
        librosa.feature.spectral_contrast(y=audio_signal[0],sr=audio_signal[1])[0,0],
        librosa.feature.chroma_stft(y=audio_signal[0],sr=audio_signal[1])[0,0],
        librosa.feature.zero_crossing_rate(y=audio_signal[0])[0,0],
        librosa.feature.rms(y=audio_signal[0])[0,0],
        librosa.feature.spectral_rolloff(y=audio_signal[0])[0,0],
        np.mean(parselmouth.Sound(audio_signal[0]).to_harmonicity().values),
        np.var(librosa.feature.fourier_tempogram(y=audio_signal[0])[0,0]),
        np.var(librosa.yin(y=audio_signal[0], fmin=80, fmax=300)),
        librosa.feature.delta(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13), order=1)[0,0],
        librosa.feature.delta(librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=13), order=2)[0,0]
    ]
    labels.append(str.split(filename,'-')[0])
    audios_features.append(features)

C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=89
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=95
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=92
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=98
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=88
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=78
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\librosa\core\spectrum

In [2251]:
print(audios_features)

[[-602.07306, 13.1947335852614, 0.62142605, 0.0810546875, 0.00096060475, 3531.4453125, -97.50147813208842, 0.0, 3664.085368991853, 8.07773, -11.23221], [-631.3037, 17.38232117976675, 0.91188693, 0.06689453125, 0.00086317235, 2723.9501953125, -99.93824761638705, 0.0, 3215.5868186394628, 9.606495, -10.414896], [-631.59393, 10.340269878220383, 0.8487246, 0.07080078125, 0.00063794665, 3499.1455078125, -93.42464877718481, 0.0, 3699.933341318098, 2.0713465, -10.990951], [-637.6097, 5.8993149032443135, 0.73967844, 0.0439453125, 0.0006119123, 2164.0869140625, -97.41982437958151, 0.0, 4257.294368115494, 8.415105, -7.584055], [-611.141, 7.745774704055531, 0.49196237, 0.0693359375, 0.0007782843, 3273.046875, -102.7743660972072, 0.0, 4062.5492286974704, 5.0149426, -10.864196], [-670.2042, 18.793902685642003, 0.0, 0.0, 0.0, 0.0, -65.07539578170545, 0.0, 3898.411032822778, 16.758188, 0.19047196], [-662.89264, 14.933015375943555, 0.0, 0.0, 0.0, 0.0, -70.3653957359376, 0.0, 2997.3693249318067, 13.5059

Crear el DataFrame

In [2253]:
df = pd.DataFrame(audios_features,columns=["MFFC","spectral_contrast","chroma_stft","zero_crossing_rate","rms","Spectral Rolloff","Harmonic Sound","Fourier","f0","MFCC Velocity","MFCC Acceleration"])
df["Label"]=labels
df = df[["Label"]+list(df.columns[:-1])]
df.to_csv("audio_features.csv",index=False)

Establecer X y Y

In [2255]:
x = df[["MFFC","spectral_contrast","chroma_stft","zero_crossing_rate","rms","Spectral Rolloff","Harmonic Sound","Fourier","f0","MFCC Velocity","MFCC Acceleration"]]
y = df["Label"]

Normalizar datos (Usando Standard Scaler)

In [2299]:
scaler = MinMaxScaler(feature_range=(-2,2))
x_scaled = scaler.fit_transform(x)

In [2285]:
pca = PCA(n_components=6)
x_pca=pca.fit_transform(x_scaled)
df_pca=pd.DataFrame(x_pca)
print(x_pca)

[[  25.40305168  -19.72380732   -9.24078347    3.16123576    2.12576951
   -15.36987217]
 [  22.47432664  -71.43823423  -10.6744476    37.2767075    -0.1357157
   -27.29087341]
 [   1.87595485  -62.96705547   -7.87757509   -6.06676625    8.57185842
   -17.90076636]
 ...
 [   1.19808071  -23.55065806 -108.50251264   31.36207491   -5.3474582
    40.37806262]
 [   7.17205266  -20.40192874  -96.72138689    3.22173294  -22.95689845
     3.87961572]
 [  10.78653764   17.47098921  -94.18892267   23.29226347  -31.58476693
     6.51969558]]


In [2301]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2, random_state=42)

Crear y entrenar el modelo (SVM)

In [2399]:
clf = SVC(kernel='rbf', C=20, gamma='scale')
clf.fit(x_train,y_train)

SVC(C=20)

In [2401]:
accuracy = clf.score(x_test,y_test)
print(accuracy)

0.4444444444444444


In [2315]:
cv_scores = cross_val_score(clf,x,y,cv=5)

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [2317]:
print(cv_scores)

[0.44444444 0.51388889 0.52777778 0.3943662  0.49295775]


Crear y entrenar el modelo (Random Forest)

In [2371]:
model = RandomForestClassifier(n_estimators=100, max_depth=32, random_state=42)
model.fit(x_train,y_train)

RandomForestClassifier(max_depth=32, random_state=42)

In [2373]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.625
